In [54]:
import numpy     
import scipy.ndimage
import scipy.special

In [55]:
class neuralNetwork:
    
    def __init__(self, inputnodes, hiddennodes, outputnodes, learningrate):
        
        self.inodes = inputnodes
        self.hnodes = hiddennodes
        self.onodes = outputnodes
        
        self.lr = learningrate
        
        #hardcoding the activation function as the sigmoid function
        self.activation_function = lambda x: scipy.special.expit(x)
        
        #We initialise weights by sampling from a normal probability distribution centred around 0 and with
        #a standard deviation that is related to the number of incoming links into a node 1/sqrt(num of incoming links)
        #Matrix to represent weights going from input nodes to hidden nodes 
        self.wih = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.hnodes, self.inodes))
        #Matrix to represent weights going from hidden nodes to output nodes 
        self.who = numpy.random.normal(0.0, pow(self.inodes, -0.5), (self.onodes, self.hnodes))
        
        
        pass
    
    
    def train(self, input_list, target_list):
        inputs = numpy.array(input_list, ndmin=2).T
        targets = numpy.array(target_list, ndmin=2).T
        
        #signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #singals outputted from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        #signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #signals outputted from final layer
        final_outputs = self.activation_function(final_inputs)
        
        #error of target - actual value
        output_errors = targets - final_outputs
        #hidden layer error is output_errors, split by weights, recombined at hidden nodes
        hidden_errors = numpy.dot(self.who.T, output_errors)
        
        #update weights for links between hidden and output layers
        self.who += self.lr * numpy.dot((output_errors * final_outputs * (1.0 - final_outputs)), numpy.transpose(hidden_outputs))
        
        #update weights for the links between the input and hidden layers
        self.wih += self.lr * numpy.dot((hidden_errors * hidden_outputs * (1.0 - hidden_outputs)), numpy.transpose(inputs))
        
        pass
    
    def query(self, inputs_list):
        inputs = numpy.array(inputs_list, ndmin=2).T
        
        #signals into hidden layer
        hidden_inputs = numpy.dot(self.wih, inputs)
        #singals outputted from hidden layer
        hidden_outputs = self.activation_function(hidden_inputs)
        #signals into final output layer
        final_inputs = numpy.dot(self.who, hidden_outputs)
        #signals outputted from final layer
        final_outputs = self.activation_function(final_inputs)
        
        return final_outputs
    
    

In [56]:
input_nodes = 784
hidden_nodes = 200
output_nodes = 10

learning_rate = 0.01

n = neuralNetwork(input_nodes, hidden_nodes, output_nodes, learning_rate)

In [57]:
#load mnist
training_data_file = open("mnist/mnist_train.csv", 'r')
training_set = training_data_file.readlines()
training_data_file.close()

In [58]:
# train the neural network

# epochs is the number of times the training data set is used for training
epochs = 6

for e in range(epochs):
    # go through all records in the training data set
    for record in training_set:
        # split the record by the ',' commas
        all_values = record.split(',')
        # scale and shift the inputs
        inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
        # create the target output values (all 0.01, except the desired label which is 0.99)
        targets = numpy.zeros(output_nodes) + 0.01
        # all_values[0] is the target label for this record
        targets[int(all_values[0])] = 0.99
        n.train(inputs, targets)
        
        ## create rotated variations
        # rotated anticlockwise by x degrees
        inputs_plusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), 10, cval=0.01, order=1, reshape=False)
        n.train(inputs_plusx_img.reshape(784), targets)
        # rotated clockwise by x degrees
        inputs_minusx_img = scipy.ndimage.interpolation.rotate(inputs.reshape(28,28), -10, cval=0.01, order=1, reshape=False)
        n.train(inputs_minusx_img.reshape(784), targets)
        
        pass
    pass

In [59]:
test_data_file = open("mnist/mnist_test.csv", 'r')
test_set = test_data_file.readlines()
test_data_file.close()

In [60]:
# test the neural network

# scorecard for how well the network performs, initially empty
scorecard = []

# go through all the records in the test data set
for record in test_set:
    # split the record by the ',' commas
    all_values = record.split(',')
    # correct answer is first value
    correct_label = int(all_values[0])
    # scale and shift the inputs
    inputs = (numpy.asfarray(all_values[1:]) / 255.0 * 0.99) + 0.01
    # query the network
    outputs = n.query(inputs)
    # the index of the highest value corresponds to the label
    label = numpy.argmax(outputs)
    # append correct or incorrect to list
    if (label == correct_label):
        # network's answer matches correct answer, add 1 to scorecard
        scorecard.append(1)
    else:
        # network's answer doesn't match correct answer, add 0 to scorecard
        scorecard.append(0)
        pass
    
    pass

In [61]:
scorecard_array = numpy.asarray(scorecard)
print ("performance = ", scorecard_array.sum() / scorecard_array.size)

performance =  0.9701
